In [22]:
# Version 2
# Manual loading images, image augmentation - image data generator
# Similiar network structure but with more filters in both convolutional layers
# CHANGE : adding little data augmentation - Rotation range 2 degrees, shifts - 10%, no flipping
# CONCLUSION : more of the almost the same images doesn't really help (too little agumentation)

In [23]:
#all imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
#import cv2
import csv
import imageio

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras import backend as K


In [24]:
K.set_image_dim_ordering('th')
seed = 7
np.random.seed(seed)

In [25]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

def readimagesfromfolder(path, color, show):
	imagepackage_list = []
	imagepackage = []
	imagelist = []
	filelist = sorted(os.listdir(path))
	for filename in filelist:
		fullfilename =  path + filename
		#print(fullfilename)        
		singleimage = imageio.imread(fullfilename)
		singleimage = rgb2gray(singleimage)
		#print(singleimage)        
		if show==1:
			plt.imshow(singleimage)
			plt.show()
		imagepackage.append(singleimage)
		imagepackage_list.append(imagepackage)        
		imagepackage = []        
	#print(len(imagepackage))        
	imagepackage_list = np.float32(imagepackage_list)
	#cv2.destroyAllWindows()
	return imagepackage_list

def readlabelfromcsv(filename):
    training_y = []
    with open(filename) as f:
        #training_y.append(f.readline())
        content = csv.reader(f)
        for line in content:
            #print(line[0])
            training_y.append(int(line[0]))
    return training_y




In [26]:
       
folderpath = 'data/basic/training/'
X_train=readimagesfromfolder(folderpath, 0, 0)
print(np.shape(X_train))

folderpath = 'data/basic/test/'
X_test=readimagesfromfolder(folderpath, 0, 0)
print(np.shape(X_test))

y_train = readlabelfromcsv("data/basic/training.csv")
y_test = readlabelfromcsv("data/basic/test.csv")

print(y_train)
print(np.shape(y_train))

print(y_test)
print(np.shape(y_test))

datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=2,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False) # randomly flip images

datagen.fit(X_train)

(162, 1, 180, 180)
(48, 1, 180, 180)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
(162,)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
(48,)


In [27]:


# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [28]:
def larger_model():
	# create model
	model = Sequential()
	model.add(Convolution2D(24, 5, 5, border_mode='valid', input_shape=(1, 180, 180), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Convolution2D(16, 3, 3, activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.5))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(50, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

# build the model
model = larger_model()
# Fit the model
model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),samples_per_epoch = 5000, nb_epoch = 20, verbose=2, validation_data=(X_test, y_test))
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print scores
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Epoch 1/20
22s - loss: 0.6655 - acc: 0.7096 - val_loss: 0.6887 - val_acc: 0.6875
Epoch 2/20
22s - loss: 0.2994 - acc: 0.8909 - val_loss: 0.7972 - val_acc: 0.6875
Epoch 3/20
22s - loss: 0.1763 - acc: 0.9367 - val_loss: 1.0144 - val_acc: 0.6875
Epoch 4/20
22s - loss: 0.1151 - acc: 0.9578 - val_loss: 1.4544 - val_acc: 0.6458
Epoch 5/20
22s - loss: 0.1021 - acc: 0.9665 - val_loss: 0.9572 - val_acc: 0.6667
Epoch 6/20
22s - loss: 0.0392 - acc: 0.9885 - val_loss: 1.5113 - val_acc: 0.6667
Epoch 7/20
22s - loss: 0.0256 - acc: 0.9934 - val_loss: 1.5702 - val_acc: 0.7500
Epoch 8/20
22s - loss: 0.0176 - acc: 0.9948 - val_loss: 1.2401 - val_acc: 0.7500
Epoch 9/20
22s - loss: 0.0190 - acc: 0.9940 - val_loss: 1.4541 - val_acc: 0.7083
Epoch 10/20
22s - loss: 0.0132 - acc: 0.9954 - val_loss: 1.8448 - val_acc: 0.6667
Epoch 11/20
22s - loss: 0.0097 - acc: 0.9966 - val_loss: 1.6931 - val_acc: 0.6667
Epoch 12/20
22s - loss: 0.0454 - acc: 0.9863 - val_loss: 1.3753 - val_acc: 0.7083
Epoch 13/20
22s - loss: 0